In [1]:
!pip uninstall -y langchain langchain-core langchain-community langchain-google-genai
!pip install langchain==0.1.0 langchain-community==0.0.10 langchain-core==0.1.8 langchain-google-genai==0.0.6 google-search-results

Found existing installation: langchain 0.1.0
Uninstalling langchain-0.1.0:
  Successfully uninstalled langchain-0.1.0
Found existing installation: langchain-core 0.1.8
Uninstalling langchain-core-0.1.8:
  Successfully uninstalled langchain-core-0.1.8
Found existing installation: langchain-community 0.0.10
Uninstalling langchain-community-0.0.10:
  Successfully uninstalled langchain-community-0.0.10
Found existing installation: langchain-google-genai 0.0.6
Uninstalling langchain-google-genai-0.0.6:
  Successfully uninstalled langchain-google-genai-0.0.6
  Using cached langchain-0.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.10-py3-none-any.whl.metadata (7.3 kB)
  Using cached langchain_core-0.1.8-py3-none-any.whl.metadata (4.0 kB)
  Using cached langchain_google_genai-0.0.6-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain-0.1.0-py3-none-any.whl (797 kB)
Using cached langchain_community-0.0.10-py3-none-any.whl (1.5 MB)
Using cached langchain_core-

In [2]:
import os
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities import SerpAPIWrapper

In [4]:
# --- CONFIGURATION ---
# # 1. Google Gemini Key
os.environ["GOOGLE_API_KEY"] = ""

# # 2. SerpAPI Key (Get free at serpapi.com)
os.environ["SERPAPI_API_KEY"] = ""

# --- CHECK KEYS ---
if not os.environ.get("GOOGLE_API_KEY"):
    print("⚠️ STOP: Please enter your Google API Key!")
elif not os.environ.get("SERPAPI_API_KEY"): 
    print("⚠️ STOP: Please enter your SerpAPI Key!")
else:
    try:
        # --- DEFINE TOOLS ---
        # 1. Real Web Search
        search = SerpAPIWrapper()

        @tool
        def search_web(query: str):
            """Useful for searching the internet for current events, live stock prices"""
            print(f"    🔎 [Search Tool] Googling: '{query}'")
            return search.run(query)

        # 2. Calculator
        @tool
        def calculator(expression: str):
            """Useful for calculating math expressions."""
            print(f"    🧮 [Math Tool] Calculating: '{expression}'")
            try:
                return str(eval(expression))
            except:
                return "Error"

        tools = [search_web, calculator]

        # --- INITIALIZE AGENT ---
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Using generic pro model for compatibility

        # The "Zero Shot ReAct" agent is the standard "Think-Act-Observe" loop
        agent_executor = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True # Helps if the LLM makes a formatting mistake
        )

        # --- RUN TASK ---
        print("\n--- 🤖 REAL AGENT TASK STARTED ---")
        query = "I have $5000 to spend how can i do it in 50/50 between aapl and nvidia(round it to the nearest whole number)."

        # In older versions, we use .run() instead of .invoke()
        response = agent_executor.run(query)

        print(f"\n--- 🏁 FINAL ANSWER ---\n{response}")

    except Exception as e:
        print(f"\n❌ Error: {e}")



--- 🤖 REAL AGENT TASK STARTED ---


> Entering new AgentExecutor chain...


E0000 00:00:1766033932.618472 2128299 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Action: search_web
Action Input: AAPL stock price    🔎 [Search Tool] Googling: 'AAPL stock price'

Observation: {'type': 'finance_results', 'title': 'News', 'exchange': 'NASDAQ', 'stock': 'AAPL', 'currency': 'USD', 'price': 271.84}
Thought:Action: search_web
Action Input: NVDA stock price    🔎 [Search Tool] Googling: 'NVDA stock price'

Observation: {'type': 'finance_results', 'title': 'News', 'exchange': 'NASDAQ', 'stock': 'NVDA', 'currency': 'USD', 'price': 170.94}
Thought:Action: calculator
Action Input: 5000 / 2    🧮 [Math Tool] Calculating: '5000 / 2'

Observation: 2500.0
Thought:Action: calculator
Action Input: 2500 / 271.84    🧮 [Math Tool] Calculating: '2500 / 271.84'

Observation: 9.196586227192467
Thought:Action: calculator
Action Input: 2500 / 170.94    🧮 [Math Tool] Calculating: '2500 / 170.94'

Observation: 14.625014625014625
Thought:Action: calculator
Action Input: round(9.196586227192467)    🧮 [Math Tool] Calculating: 'round(9.196586227192467)'

Observation: 9
Thought:Ac

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 39.999364898s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 5


I now know the final answer
Final Answer: You can buy 9 shares of AAPL and 15 shares of NVDA.

> Finished chain.

--- 🏁 FINAL ANSWER ---
You can buy 9 shares of AAPL and 15 shares of NVDA.
